In [1]:
#http://www.doganaskan.com/trendypy/source/seeinaction.html

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.preprocessing import StandardScaler


In [2]:
df=pd.read_csv("df18_ts.csv")
date=pd.to_datetime(df.DATE)
df["DATE"]=date
df.head()

,Account_No,DATE,index,year,month,day,withd_by_date,dep_by_date,Balance_updated
0,1196428',2018-01-01,15391,2018,1,1,35000000.00,1408920.67,-1.322909e+08
1,1196428',2018-01-02,15410,2018,1,2,59400000.00,452336.73,-1.322153e+08
2,1196428',2018-01-03,15437,2018,1,3,43502723.23,724458.09,-1.224491e+08
3,1196428',2018-01-04,15463,2018,1,4,40300000.00,772555.00,-1.268383e+08
4,1196428',2018-01-05,15489,2018,1,5,44001591.01,1128626.00,-1.265977e+08


In [3]:
df_withdrawal=df[['Account_No','DATE','withd_by_date']]
df_withdrawal.sort_values(by='DATE',inplace=True)

C:\Users\E330\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [4]:
clients=list(df_withdrawal["Account_No"].unique())

In [5]:
clients_data=[]
for acc in clients:
    client0=df_withdrawal[df_withdrawal["Account_No"]==acc]
    client0.rename({'withd_by_date': acc}, axis=1, inplace=True)
    client0.drop(["Account_No"], axis=1, inplace =True)
    client0=pd.DataFrame(client0, columns=['DATE',acc])
    client0.set_index('DATE',inplace=True)
    clients_data.append(client0)
# clients_data
# client1=df_withdrawal[df_withdrawal["Account_No"]==cluster_accounts[1]]
# client1.rename({'withd_by_date': cluster_accounts[1]}, axis=1, inplace=True)
# client1.drop(["Account_No"], axis=1, inplace =True)
# clients_data.append(client1)
# clients_data
# withd_series=pd.concat([clients_data[0].set_index('DATE'),clients_data[1].set_index('DATE')], axis = 1)

withd_series=pd.concat(clients_data, axis=1, sort=False)
# withd_series=withd_series.add_suffix('_1')
withd_series.reset_index(inplace=True, drop=False)
withd_series.rename({'index': 'date'}, axis=1, inplace=True)
# withd_series.reset_index(inplace=True, drop=False)
withd_series.rename({'DATE': 'date'}, axis=1, inplace=True)

withd_series.fillna(0, inplace=True)

withd_series["date"]= withd_series["date"].astype(str) 
withd_series["date"].replace("-","", regex=True, inplace=True)



C:\Users\E330\Anaconda3\lib\site-packages\pandas\core\frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)
C:\Users\E330\Anaconda3\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [8]:
work_df=withd_series.copy()
work_df.drop(["date"], inplace=True, axis=1)

In [9]:
#Standarize features values to apply clustering methods
def df_clustering_std(df_clustering):
    data=df_clustering.copy()
    features=list(data.columns)
#     features=["1196711'_1", "409000493210'_1", "409000425051'_1",  "409000405747'_1"]
    x = data.loc[:, features].values
    #Standardizing the features
    x = StandardScaler().fit_transform(x)
    data[features] =x
    return data

In [10]:
work_df=df_clustering_std(work_df)

In [11]:
work_df.head()

,1196428',409000362497',409000438611',409000438620',409000493201',409000611074',A,B,1196711',409000493210',409000425051',409000405747'
0,-0.232276,1.607083,-0.520616,1.954297,0.110540,-0.985670,2.861790,1.730238,-0.125083,-0.101766,-0.059299,-0.103754
1,0.801706,1.333390,-1.013153,0.133854,-0.307230,-0.706440,-0.351018,0.048017,-0.125083,-0.101766,-0.059299,-0.103754
2,0.128038,1.096081,-0.289724,0.900443,-0.094572,-0.086319,-0.351018,-0.340188,-0.095602,-0.101766,-0.059299,-0.103754
3,-0.007682,1.298311,-0.608575,0.947740,-0.166795,-0.590872,-0.351018,-0.340188,-0.125083,-0.101766,-0.059299,-0.103754
4,0.149178,1.654843,-0.663440,0.826743,-0.133034,-0.441127,1.523120,-0.340188,-0.125083,-0.101766,-0.059299,-0.103754


In [12]:
# !pip install trendypy
from trendypy.trendy import Trendy

trendy = Trendy(n_clusters=5) # 2 for tech and banking

trendy.fit([work_df[i] for i in work_df.columns])

trendy.labels_

[0, 0, 1, 0, 3, 2, 3, 3, 4, 4, 4, 4]

In [21]:
clusters=pd.DataFrame()
clusters['Accounts']=work_df.columns
clusters['trendy_label']=trendy.labels_

In [37]:
cluster_accounts=list(clusters[clusters["trendy_label"]==4].Accounts)
cluster_data2=work_df[cluster_accounts]
cluster_data2=cluster_data2.add_suffix('_2')
cluster_data2.head()

,1196711'_2,409000493210'_2,409000425051'_2,409000405747'_2
0,-0.125083,-0.101766,-0.059299,-0.103754
1,-0.125083,-0.101766,-0.059299,-0.103754
2,-0.095602,-0.101766,-0.059299,-0.103754
3,-0.125083,-0.101766,-0.059299,-0.103754
4,-0.125083,-0.101766,-0.059299,-0.103754


In [38]:
cluster_data1=withd_series[cluster_accounts]
cluster_data1=cluster_data1.add_suffix('_1')
# cluster_data1["date"]=withd_series["date"]
cluster_data1.head()

,1196711'_1,409000493210'_1,409000425051'_1,409000405747'_1
0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0
2,800000.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0


In [39]:
withd_series_all=pd.concat([withd_series["date"],cluster_data1,cluster_data2], axis=1)


In [40]:
withd_series_all.head()

,date,1196711'_1,409000493210'_1,409000425051'_1,409000405747'_1,1196711'_2,409000493210'_2,409000425051'_2,409000405747'_2
0,20180101,0.0,0.0,0.0,0.0,-0.125083,-0.101766,-0.059299,-0.103754
1,20180102,0.0,0.0,0.0,0.0,-0.125083,-0.101766,-0.059299,-0.103754
2,20180103,800000.0,0.0,0.0,0.0,-0.095602,-0.101766,-0.059299,-0.103754
3,20180104,0.0,0.0,0.0,0.0,-0.125083,-0.101766,-0.059299,-0.103754
4,20180105,0.0,0.0,0.0,0.0,-0.125083,-0.101766,-0.059299,-0.103754


In [41]:
withd_series_all.to_csv("cluster_data4.csv", index = None, header=True)